### 반복량 축소

In [4]:
import pandas as pd
import random
import time

# 테스트 데이터셋 설명
# df_a : {user_id, some_data} 두 컬럼으로 이루어져있고, some_data 는 약 20%의 확률로 NaN
# df_b : df_a 와 컬럼이 같고, 동일한 user_id 값들을 가지고 있다. alt_data 는 NaN이 없다.  
# 목표 : df_a 의 NaN 인 some_data 값을 찾아 df_b 의 alt_data 로 대체
## merge 등 더 효율적인 방법은 다음 포스팅에서 살펴본다.

# 데이터셋 준비
qnt = 10000 # 데이터 수
df_a_origin = pd.DataFrame({
    "user_id" : [str(x) for x in range(1, qnt+1)],
    "some_data" : [random.randint(0, qnt) if random.randint(0, qnt) < qnt*4/5 else None
                   for _ in range(qnt)]
})
df_b = pd.DataFrame({
    "user_id" : [str(x) for x in range(1, qnt+1)],
    "alt_data" : [random.randint(0, qnt) for _ in range(qnt)]
})

# 대조군
# (1) : df_a 의 각 row 를 순환하면서 NaN인 some_data가 있는 row를 찾는다. -- O(N_a)
# (2) : NaN 값을 찾았다면, df_b 에서 해당 값의 user_id 에 해당하는 alt_data 를 찾는다. -- O(N_b)
# (3) : df_a의 some_data 를 alt_data 로 치환한다. -- O(1)
# - 전체 O(N_a * N_b) 정도의 시간 복잡도를 가진다.
# - 또한 boolean mask를 생성하므로 → 메모리 비용이 발생한다.
df_a = df_a_origin.copy()
start = time.time()
for i, row in df_a.iterrows():
    if pd.isna(row["some_data"]):
        df_a.at[i, "some_data"] = df_b.loc[df_b["user_id"] == row["user_id"], "alt_data"].iloc[0]
print(f"time check 1 ::: {time.time() - start}")

# 실험군
# (1) some_data가 NaN 인 idx를 미리 뽑아놓는다. (10000개 > 약 2000개) -- O(N_a)
# (2) df_a 의 some_data 가 NaN 인 row 만 뽑아 순환한다. -- O(K) (K≈0.2*N_a)
# (3) 각 루프마다 df_b 에서 해당 값의 user_id 에 해당하는 alt_data 를 찾는다. -- O(N_b)
# (4) : df_a의 some_data 를 alt_data 로 치환한다. -- O(1)
# - 전체 시간 복잡도는 O(N_a) + O(K*N_b) (K≈0.2*N_a) 가량이 된다. (실제 빅오 표기로는 O(K*N_b))
# - 의의 : 반복 횟수를 줄여 시간 복잡도상 실수행 배수를 줄여 실행시간을 줄인다.
df_a = df_a_origin.copy()
start = time.time()
target_idx = df_a[df_a["some_data"].isna()].index
for i, row in df_a.loc[target_idx].iterrows():
    df_a.at[i, "some_data"] = df_b.loc[df_b["user_id"] == row["user_id"], "alt_data"].iloc[0]
print(f"time check 2 ::: {time.time() - start}")

# 리뷰
# 성능이 소폭 향상됐다. 하지만 큰 향상은 보이지 못하고 있다.
# 이는, 가장 큰 병목현상을 보이는 구간은 df_b 에서 user_id 를 찾는 부분이기 때문이다.
# 다음 포스팅에서는 이를 해소하기 위한 좋은 활용 사례를 찾아본다.

# 참고 : boolean mask
# 데이터와 같은 길이를 가지는 True / False 값의 배열(또는 Series) 
# e.g. mask = df["some_data"].isna()
# 위 예시에서는 루프마다 df_b에 대한 user_id 를 찾는 부분에서 boolean mask 를 생성하므로, 비효율적인 연산이 수행된다.
# (이 예시에서는 boolean_mask 생성의 목적 = df_b 를 스캔해서 user_id 에 해당하는 행을 찾음)

time check 1 ::: 0.7818460464477539
time check 2 ::: 0.6828429698944092


### 딕셔너리 캐싱  

In [1]:
import pandas as pd
import random
import time

# 테스트 데이터셋 설명
# df_a : {user_id, some_data} 두 컬럼으로 이루어져있고, some_data 는 약 20%의 확률로 NaN
# df_b : df_a 와 컬럼이 같고, 동일한 user_id 값들을 가지고 있다. alt_data 는 NaN이 없다.  
# 목표 : df_a 의 NaN 인 some_data 값을 찾아 df_b 의 alt_data 로 대체
## merge 등 더 효율적인 방법은 다음 포스팅에서 살펴본다.

# 데이터셋 준비
qnt = 10000 # 데이터 수
df_a_origin = pd.DataFrame({
    "user_id" : [str(x) for x in range(1, qnt+1)],
    "some_data" : [random.randint(0, qnt) if random.randint(0, qnt) < qnt*4/5 else None
                   for _ in range(qnt)]
})
df_b = pd.DataFrame({
    "user_id" : [str(x) for x in range(1, qnt+1)],
    "alt_data" : [random.randint(0, qnt) for _ in range(qnt)]
})

# 대조군
# (1) : df_a 의 각 row 를 순환하면서 NaN인 some_data가 있는 row를 찾는다. -- O(N_a)
# (2) : NaN 값을 찾았다면, df_b 에서 해당 값의 user_id 에 해당하는 alt_data 를 찾는다. -- O(N_b)
# (3) : df_a의 some_data 를 alt_data 로 치환한다. -- O(1)
# - 전체 O(N_a * N_b) 정도의 시간 복잡도를 가진다.
# - 또한 boolean mask를 생성하므로 → 메모리 비용이 발생한다.
df_a = df_a_origin.copy()
start = time.time()
for i, row in df_a.iterrows():
    if pd.isna(row["some_data"]):
        df_a.at[i, "some_data"] = df_b.loc[df_b["user_id"] == row["user_id"], "alt_data"].iloc[0]
print(f"time check 1 ::: {time.time() - start}")

# 실험군 1 : 딕셔너리 캐싱  
# (1) df_b 를 dictionary로 변환해 캐싱 > b_dict -- O(N_b)
# (2) df_a 를 순환하면서 -- O(N_a)
# (3) row  의 user_id 가 NaN인 경우를 찾아
# (4) b_dict 에서 user_id 를 찾는다. -- O(1)
# (5) df_a 의 some_data 를 b_dict 의 value 값으로 대체한다.
# 전체 시간 복잡도 : O(N_b) + O(N_a)
# 의의 : 각 루프에서 수행되는 df_b["user_id"] 즉, O(n)의 시간복잡도 연산을 캐싱을 통해 O(1) 으로 줄였다.
# 즉, 매 루프마다 boolean mask 를 만드는 작업(=스캔)을 제거했다.
# 원리 : 딕셔너리는 key 기반으로 해시테이블에 정리가 되어있음 -> O(1)의 시간복잡도를 가짐
df_a = df_a_origin.copy()
start = time.time()
b_dict = {uid : row for uid, row in df_b.set_index("user_id").iterrows()}
for i, row in df_a.iterrows():
    if (pd.isna(row["some_data"])):
        df_a.at[i, "some_data"] = b_dict[row["user_id"]]["alt_data"]
print(f"time check 2 ::: {time.time() - start}")

# 실험군 2 : 딕셔너리 캐싱 + 반복량 축소
# 앞선 포스팅에서 살펴본 반복 횟수 감소소 기법도 더해서 실험해보면
df_a = df_a_origin.copy()
start = time.time()
target_idx = df_a[df_a["some_data"].isna()].index
b_dict = {uid : row for uid, row in df_b.set_index("user_id").iterrows()}
for i, row in df_a.loc[target_idx].iterrows():
    df_a.at[i, "some_data"] = b_dict[row["user_id"]]["alt_data"]
print(f"time check 3 ::: {time.time() - start}")

# 원리
# 캐싱 : 자주 쓰는 값을 미리 준비해두는 것  
# 딕셔너리 해시 : 내부적으로 해시(hash) 라는 알고리즘을 사용해 key를 숫자로 바꾸고, 그 숫자를 이용해 저장 위치를 바로 찾음
# 캐싱 = "필요한 값을 미리 저장해두는 행위", 딕셔너리 해시 = "미리 지정된 값들에 O(1)로 접근하게 해주는 데이터 구조"
# 다음에는 가장 효율적인 방식으로, 해시 기반 join 을 이용해 반복문을 아예 제거하는 방법을 살펴본다.

# 추가 정리
# pd.isna() -- 어떤 값 자체를 NaN인지 확인하는 경우

time check 1 ::: 0.8492498397827148
time check 2 ::: 0.21245098114013672
time check 3 ::: 0.11246085166931152


In [6]:
print(hash("some_data"))
print(hash("alt_data"))

-2155848453315933556
7091118814780763299


In [65]:
# 구체적으로 뜯어보기 - 기존방식
df_a = df_a_origin.copy()
start = time.time()
# (1) df_a를 돌면서 -> 비용 = O(N_a)
for i, row in tqdm(df_a.iterrows()):
    if pd.isna(row["some_data"]):
        # (2) df_b를 돌면서 user_id를 찾음 -> 비용 = O(N_b)
        df_a.at[i, "some_data"] = df_b.loc[df_b["user_id"] == row["user_id"], "alt_data"].iloc[0]
print(f"time check 1 ::: {time.time() - start}")
##### ==> 총 비용 : O(N_a) * O(N_b) / a, b 대략 1만건 씩이면 = 총 1억회 연산


# 구체적으로 뜯어보기 1
df_a = df_a_origin.copy()
time_check = time.time()
# (1) df_b 를 돌면서 dict를 만든다 -> 비용 = O(N_b)
b_dict = {uid : row for uid, row in df_b.set_index("user_id").iterrows()}
print(f"time check 1 ::: {time.time() - time_check}")
# (2) df_a 를 돌면서 -> 비용 = O(N_a)
time_check = time.time()
for i, row in tqdm(df_a.iterrows()):
    # (3) 해시테이블에 바로 접근해서 값을 가져옴 -> 비용 = O(1) 에 근접
    df_a.at[i, "some_data"] = b_dict[row["user_id"]]["alt_data"]
print(f"time check 2 ::: {time.time() - time_check}")
##### ==> 총 비용 : O(N_b) + O(N_a) * O(1) / a, b 대략 1만건 씩이면 = 총 20000 회 연산 (에 근접)


# 구체적으로 뜯어보기 2
df_a = df_a_origin.copy()
time_check = time.time()
# (1) df 를 돌면서 dict를 만든다 -> 비용 = O(N_b)
b_dict = {uid : row for uid, row in df_b.set_index("user_id").iterrows()}
# (2) df 를 돌면서 na 인덱스 식별 -> 비용 = O(N_a) 
target_idx = df_a[df_a["some_data"].isna()].index
print(f"time check 1 ::: {time.time() - time_check}")
time_check = time.time()
# (3) 필터링된 df 를 돌면서 -> 비용 = O(N_a * 0.2)  // 위에서 NULL 비율을 대략 0.2로 잡음
for i, row in tqdm(df_a.iloc[target_idx].iterrows()):
    # (4) 해시테이블에 바로 접근해서 값을 가져옴 -> 비용 = O(1) 에 근접
    df_a.at[i, "some_data"] = b_dict[row["user_id"]]["alt_data"]
print(f"time check 2 ::: {time.time() - time_check}")
##### ==> 총 비용 : O(N_b) + O(N_a)  +  O(N_a*0.2) * O(1) / a, b 대략 1만건 씩이면 = 총 22000회 연산
##### 그런데 왜 2번 방식보다 빠르냐? 이것은 바로..!
##### na 를 식별하는 isna() 연산이 "벡터연산"으로 굉장히 빠르기 때문

10000it [00:01, 6133.02it/s]


time check 1 ::: 1.6339082717895508
time check 1 ::: 0.283344030380249


10000it [00:00, 18258.66it/s]


time check 2 ::: 0.5476853847503662
time check 1 ::: 0.2526881694793701


2037it [00:00, 18197.07it/s]

time check 2 ::: 0.11291956901550293


### 해시 기반 join을 활용한 반복문 제거와 데이터프레임 연산 성능 최적화

In [7]:
import pandas as pd
import random
import time

# 테스트 데이터셋 설명
# df_a : {user_id, some_data} 두 컬럼으로 이루어져있고, some_data 는 약 20%의 확률로 NaN
# df_b : df_a 와 컬럼이 같고, 동일한 user_id 값들을 가지고 있다. alt_data 는 NaN이 없다.  
# 목표 : df_a 의 NaN 인 some_data 값을 찾아 df_b 의 alt_data 로 대체
## merge 등 더 효율적인 방법은 다음 포스팅에서 살펴본다.

# 데이터셋 준비
qnt = 10000 # 데이터 수
df_a_origin = pd.DataFrame({
    "user_id" : [str(x) for x in range(1, qnt+1)],
    "some_data" : [random.randint(0, qnt) if random.randint(0, qnt) < qnt*4/5 else None
                   for _ in range(qnt)]
})
df_b = pd.DataFrame({
    "user_id" : [str(x) for x in range(1, qnt+1)],
    "alt_data" : [random.randint(0, qnt) for _ in range(qnt)]
})

# 대조군
# (1) : df_a 의 각 row 를 순환하면서 NaN인 some_data가 있는 row를 찾는다. -- O(N_a)
# (2) : NaN 값을 찾았다면, df_b 에서 해당 값의 user_id 에 해당하는 alt_data 를 찾는다. -- O(N_b)
# (3) : df_a의 some_data 를 alt_data 로 치환한다. -- O(1)
# - 전체 O(N_a * N_b) 정도의 시간 복잡도를 가진다.
# - 또한 boolean mask를 생성하므로 → 메모리 비용이 발생한다.
df_a = df_a_origin.copy()
start = time.time()
for i, row in df_a.iterrows():
    if pd.isna(row["some_data"]):
        df_a.at[i, "some_data"] = df_b.loc[df_b["user_id"] == row["user_id"], "alt_data"].iloc[0]
print(f"time check 1 ::: {time.time() - start}")

# 실험군
# (1) df_a 와 df_b 를 "user_id" 기반으로 merge -- O(N_a + N_b)
#     일반적으로 df 를 해시테이블로 구성할 때 O(n) = O(N_a)
#     그리고 상대방 df 를 한 번 순회하면서 매칭할 때 O(n) = O(N_b)
#     따라서 O(N_a) + O(N_b)
# (2) some_data 가 NaN 인 경우 Fillna -- O(?)
# - 전체 O(??)의 시간 복잡도를 가진다.
# - 의의
#   * python-level 반복문 제거
#   * 반복적인 boolean mask 생성 제거
#   * C/CPython 기반의 벡터화된 join 연산(고속) 사용
# - 원리
#   * merge 는 해시 기반 join 알고리즘을 통해 평균 O(1) 복잡도의 lookup 이 가능
#   * 전체 작업이 내부적으로 벡터화되어 있으며 C/Cython 코드로 동작
#   * 내부적으로 벡터화된 연산과 연속 메모리 접근을 활용할 수 있음
df_a = df_a_origin.copy()
start = time.time()
df_merge = df_a.merge(df_b, on="user_id", how="left") # O(N_a) + O(N_b)
df_merge["some_data"] = df_merge["some_data"].fillna(df_merge["alt_data"]) # fillna : 벡터화된 연산. O(N_a)
df_merge
print(f"time check 2 ::: {time.time() - start}")

# 원리에 대한 설명
# merge
#   * pandas 의 merge 는 내부적으로 hash 기반 join을 사용한다. 아래와 같은 작동 방식을 가진다.  
#   * (1) 한쪽 테이블(df_a)를 해시 테이블로 변겅 : O(N_a)
#   * (2) 다른쪽 테이블(df_b)를 순회하며 : O(N_b)
#   * (3) df_b 의 각 row의 key로 해시테이블에서 매칭되는 row를 lookup : O(1)
#   시간복잡도 : O(N_a) + O(N_b * 1) = O(N_a + N_b)
#   주의 : 어떤쪽을 해시 테이블로 만들지는, 데이터에 따라 다름. pandas가 둘 중 해시테이블로 만드는 게 유리한 쪽을 해시테이블로 만듦(데이터 크기, 데이터타입, 중복도 등)  
# fillna
#   * pandas 의 fillna 는 C/CPython 레벨의 벡터화된 연산을 하므로 빠르다. 아래와 같이 작동한다.  
#   * (1) df_a 의 some_data 를 한 번 순회 : O(N_a)
#   * (2) 각 순회마다 값이 NaN인지 검사 > NaN이면 alt_data 값으로 대체 : O(1)
#   시간복잡도 : O(N_a * 1) = O(N_a)
# 총 시간복잡도
#   O(N_a) + O(N_b) + O(N_a)  = O(N_a + N_b)

time check 1 ::: 0.790179967880249
time check 2 ::: 0.00737309455871582
